#### Create Public/Private label in dataset
This notebook creates the Private/Public label in the dataset.
The label is created by counting the number of times each sequence appears in the dataset.
If the sequence appears more than once, it is labeled as Public, otherwise it is labeled as Private.
The threshold for public can be changed by changing the value of the variable "threshold" in the function "label_public_private".
The output file is saved in the same directory as the input file.



In [52]:
cd ..

/home/romi/projects


In [53]:
import pandas as pd
import os
import collections
import matplotlib.pyplot as plt
from lab_notebooks.utils import TRANSFORMER, DEVICE, DATA_DIR

#### Load and Prepare Data

In [ ]:
# edit according to the desired file
data_file_name = "data_file_name.csv"

In [ ]:
# load data
data_dir = DATA_DIR + data_file_name
tcrb_data = pd.read_csv(data_dir,engine="pyarrow")
tcrb_data

In [71]:
# edit column names
tcrb_data.rename(columns={'amino_acid': 'aaSeqCDR3'}, inplace=True)
tcrb_data.rename(columns={'sample': 'SampleName'}, inplace=True)

In [ ]:
tcrb_data = tcrb_data[["aaSeqCDR3", "v_gene", "SampleName"]]
tcrb_data

In [ ]:
len(collections.Counter(tcrb_data["SampleName"])), len(tcrb_data.index)

In [ ]:
# drop duplicate rows
tcrb_data_no_duplicates = tcrb_data.drop_duplicates()
len(tcrb_data_no_duplicates.index)

In [ ]:
collections.Counter(tcrb_data_no_duplicates['SampleName'])

In [ ]:
# count number of times aaSeqCDR3 appear - each sequence appears once in each sample so the duplicate occurances
# will be in different samples
num_of_occurances_df = tcrb_data_no_duplicates['aaSeqCDR3'].value_counts().to_frame()
len(num_of_occurances_df.index)
num_of_occurances_df.head()

In [ ]:
num_of_occurances_df.index.name = 'Sequences'
num_of_occurances_df.reset_index(inplace=True)
num_of_occurances_df = num_of_occurances_df.rename(columns={'aaSeqCDR3': 'Appearances'})
num_of_occurances_df.head()

In [83]:
stats = collections.Counter(num_of_occurances_df['Appearances'])

#### Label as Public/Private

In [84]:
# if occurrence value is larger than 1, the sequence is public(1), otherwise it's private(0)
# change here for different threshold 10,50,100 if desired
def label_public_private (row):
    if row['Appearances'] > 1 : return 1
    return 0

In [ ]:
num_of_occurances_df['Private_Public_label'] = \
    num_of_occurances_df.apply(lambda row: label_public_private(row), axis=1)
num_of_occurances_df.head()

In [86]:
tcrb_data_no_duplicates = tcrb_data_no_duplicates.rename(columns={'aaSeqCDR3': 'Sequences'})
# merge the two dataframes num_of_occurances_df and tcrb_data_no_duplicates on Sequences
merged_df = pd.merge(num_of_occurances_df, tcrb_data_no_duplicates, on='Sequences')
# remove SampleName column from merged_df and drop duplicates
merged_df_no_duplicates = merged_df.drop(['SampleName'], axis=1)
merged_df_no_duplicates = merged_df_no_duplicates.drop_duplicates()
len(merged_df_no_duplicates.index)

#### Export to csv

In [90]:
# output to csv
output_path = "full_output_path/output_file_name.csv"
merged_df_no_duplicates.to_csv(output_path)